In [2]:
# import tf_sentencepiece as tfs
# import sentencepiece as spm
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as tftext
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow_text'

In [2]:
vocab_file= '/space/engineering/pretrained_models/bert/multi_cased_L-12_H-768_A-12/vocab.txt'

In [3]:
##restricting no of gpus
gpus = tf.config.list_physical_devices('GPU')
device_to_use = gpus[7]
tf.config.experimental.set_memory_growth(device_to_use,True)
tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
print(tf.config.get_visible_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]


In [12]:
# class BertTokenizationCustom(tf.keras.layers.Layer):
class BertTokenizationCustom(object):
    def __init__(self,vocab_path):
#         super(BertTokenizationCustom, self).__init__(trainable=False,dtype=tf.int32)
        
        self.missing_index=-1
        #self.vocab_path = vocab_path
        self.vocab_table = tf.lookup.StaticHashTable(
            tf.lookup.TextFileInitializer(vocab_path,
                                          key_dtype=tf.string,key_index=tf.lookup.TextFileIndex.WHOLE_LINE,
                                     value_dtype=tf.int64,value_index=tf.lookup.TextFileIndex.LINE_NUMBER),default_value=self.missing_index)
        self.max_seq_length = 10
        
    def build(self,input_shape):
        #self.model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
        ## pad_id is usually 0 
#         [self.CLS_ID,self.SEP_ID,self.PAD_ID]  = tfs.piece_to_id(['[CLS]','[SEP]','[PAD]'],model_file=self.model_path)
        self.built=True
        
#     @tf.function    
    def call(self, input_text):
        ##tensorflow sentence piece works while exporting to graph while, tf_text sentencepiece doesn't
        encoded_text = self.process_sentences(input_text)
        return encoded_text
    
    def is_vocab_missing(self,vocab_lookup):
        return vocab_lookup==self.missing_index
    
#     @tf.function
    def get_chars_len(self,chars):
        chars_len = tf.shape(chars)[0]
        # when it is ragged tensor
    #     chars_len = chars.nrows()
        return chars_len

    def inner_cond(self,start,end,word,chars,vocab_lookup):
        cond = tf.less(start,end) and self.is_vocab_missing(vocab_lookup)
#         tf.print(f'vocab lookup inside inner_cond  start {start} end {end} condition {vocab_lookup} is_vocab_missing {is_vocab_missing(vocab_lookup)} & entire cond {cond}')
        return cond
    
    # def inner_while_body(start,end,word,vocab_lookup):
    def inner_while_body(self,start,end,word,chars,vocab_lookup):
#         tf.print(f'inner_while_body start {start} end {end} word {word} chars {chars}\n')
    #     chars = tf.strings.bytes_split(word)
        substring = tf.strings.join(tf.slice(chars,begin=start,size=end-start))
    #     tf.strings.reduce_join()
        if start>0:
            substring = tf.strings.join(["##",substring])
        # assigning it rather than creating a new variable so that this is accessible to other
        # function referencing this variable
        self.vocab_lookup.assign(self.vocab_table.lookup(tf.constant(substring)))
#         tf.print(f'substring {substring} vocab lookup {vocab_lookup}')
        if self.vocab_lookup==self.missing_index:
    #         print(f'missing {substring}')
#             tf.print(f'missing {substring}')
            # subtracting only if missing
            end.assign_sub([1])
#         else:
#             tf.print(f'present {substring}')

        return (start,end,substring,chars,vocab_lookup)
    
    def outer_cond(self,start,end,word,chars,is_bad,sub_tokens):
#         tf.print(f'outer_cond start {start} end {end}  chars {chars} is_bad {is_bad} sub_tokens {sub_tokens}')
        return tf.math.logical_and(start < tf.shape(chars)[0],tf.math.logical_not(is_bad))

#     @tf.function
    def outer_while_loop(self,start,end,word,chars,is_bad,sub_tokens):
#         tf.print(f'outer_while_loop start {start} end {end}  chars {chars} is_bad {is_bad} sub_tokens {sub_tokens}')
        # vocab_lookup = vocab_table.lookup(tf.constant(substring))
#         chars_len = self.get_chars_len(chars)
        chars_len = tf.shape(chars)[0]
        # end = tf.Variable([chars_len])
        end = tf.Variable([0])
        end.assign([chars_len])
        print(f'end {end} start {start}')
        size = end-start
        sliced_text_tensor = tf.slice(chars,begin=start,size=size)
        num_or_size_splits = tf.shape(sliced_text_tensor)[0]
        print(f'sliced_text {sliced_text_tensor} shape is {tf.shape(sliced_text_tensor)} num_or_size_splits {num_or_size_splits}')
        
        #sliced_text_list = tf.split(sliced_text_tensor, num_or_size_splits = num_or_size_splits, axis = 0)
        # TODO remove
        sliced_text_list = sliced_text_tensor
        substring = tf.strings.join(sliced_text_list)
        vocab_lookup = tf.Variable(self.missing_index,shape=(),name='vocab_lookup',dtype=tf.int64)
    #     vocab_lookup.assign(vocab_table.lookup(tf.constant(substring)))

        (start_1,end_1,substring_1,chars_1,vocab_lookup) = tf.while_loop(self.inner_cond,self.inner_while_body,loop_vars=[start,end,substring,chars,vocab_lookup])
#         tf.print(f'****end of inner while start {start_1} end {end_1} substring {substring_1} chars {chars_1} vocab_lookup {vocab_lookup} \n\n')
        if not is_vocab_missing(vocab_lookup):
            sub_tokens_concat = tf.concat([sub_tokens,[substring_1]],axis=0)
#             tf.print("sub tokens concat ",sub_tokens_concat, 'shape ',tf.shape(sub_tokens_concat))
    #         sub_tokens.append(substring)
#             tf.print('subtokens ',sub_tokens, ' sub tokens shape ',sub_tokens)
            sub_tokens.assign(sub_tokens_concat)
#             tf.print(f'subtokens appended ')
        else:
            # if vocab missing even after the inner loop then this word can't be formed using
            # word existing in the dictionary
            is_bad.assign(True)
#         tf.print(f'start before assigning end {start} & end val {end}')
        start.assign(end_1)
#         tf.print(f'current subtokens {sub_tokens}')
        return (start,end,word,chars,is_bad,sub_tokens)
    
    @tf.function
    def per_word_tokenize(self,word):
        print(f'word {word}')
#         word = tf.squeeze(word)
        chars = tf.strings.bytes_split(word)#.to_tensor()
#         tf.print(f'per_word_tokenize word  {word} shape {tf.shape(word)} chars {chars}')
        chars_len = tf.shape(chars)[0]
#         chars_len = self.get_chars_len(chars)
    #     start = tf.zeros((1,),dtype=tf.int32)
        start = tf.Variable([0],dtype=tf.int32)
#         end = tf.Variable([chars_len])
        # directly initializing with a tensor 'chars_len' didn't work, TODO figure out
        end = tf.Variable([0])
        end.assign([chars_len])

        is_bad = tf.Variable(False)
        sub_tokens = tf.Variable([],shape=(None,),dtype=tf.string)

        (start,end,word,chars,is_bad,sub_tokens_1) =  tf.while_loop(self.outer_cond,self.outer_while_loop,loop_vars=[start,end,word,chars,is_bad,sub_tokens])
        sub_tokens.assign(sub_tokens_1)
    #     sub_tokens_out = tf.expand_dims(sub_tokens,axis=0)
        return sub_tokens
    def joined_per_word_tokenized(self,word):
        sub_tokens = self.per_word_tokenize(word)
        return tf.strings.reduce_join(sub_tokens,separator=' ')
    
    def process_sentences(self,sentences):
        # words is a ragged tensor of words - [batch_size,no_of_words]
        words = tf.strings.split(sentences)
        print(f'words in process sentence {words}')
    #     list_words = tf.expand_dims(words,axis=1)
    #     tf.print(f'words in process sentence {list_words} shape {tf.shape(list_words)}')
        subword_tokenized_list = tf.map_fn(self.joined_per_word_tokenized,words,parallel_iterations=None,infer_shape=False)
        
        sentence_tokenized = tf.strings.reduce_join(subword_tokenized_list,separator=' ')
        processed_tokens = tf.strings.split(sentence_tokenized)
#         tf.print(f'processed tokens {processed_tokens}')
        sequence_length = tf.shape(processed_tokens)[-1]
        trimmed_max_length = max_sequence_length-2
        values_trimmed = tf.cond(tf.greater(sequence_length,trimmed_max_length), 
                                        lambda : tf.slice(processed_tokens,begin=[0],size=[trimmed_max_length],name='trimmed_out'),lambda : processed_tokens)
        concat = tf.concat([['[CLS]'], values_trimmed, ['[SEP]']],axis=0,name='concat_out')
        actual_token_length = tf.shape(concat)[-1]
        token_ids = self.vocab_table.lookup(concat)
        padded = tf.pad(token_ids,paddings=[[0,max_sequence_length-actual_token_length]],name='input_ids')
        return padded
    
#     def process_sentences(self,sentences):
# #         sentences_tokenized = tf.map_fn(self.process_sentence,sentences)
#         sentences_tokenized = process_sentence(sentences)
#         return sentences_tokenized


In [13]:
tf.keras.backend.clear_session()

In [14]:
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
# tokenized_out = BertTokenizationCustom(vocab_path=vocab_file)(input_text)
tokenized_out = BertTokenizationCustom(vocab_path=vocab_file)
# model_lookup = tf.keras.Model(inputs={'input_text':input_text},outputs=tokenized_out)

In [15]:
tokenized_out.process_sentences(tf.constant(['talk to an agent','talk to a representative']))

NameError: name 'process_sentence' is not defined

In [18]:
temp_string = tf.constant(["hi this is jithin","talk to"])

In [19]:
tf.strings.split(temp_string)

<tf.RaggedTensor [[b'hi', b'this', b'is', b'jithin'], [b'talk', b'to']]>

In [ ]:
tf.gather()

In [ ]:
tftext.gather_with_default()

In [ ]:
tf.strided_slice()

In [30]:
temp_string = tf.constant("hi this is")

In [32]:
tf.strings.split(temp_string,sep=" ")

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'hi', b'this', b'is'], dtype=object)>

In [26]:
x = tf.Variable(tf.random.uniform([5, 30], -1, 1))

# Split `x` into 3 tensors along dimension 1
s0, s1, s2 = tf.split(x, num_or_size_splits=3, axis=1)
splitted_out = tf.split(x, num_or_size_splits=3, axis=1)


In [28]:
s0.shape

TensorShape([5, 10])

In [31]:
from tensorflow.python.training.tracking import util as trackable

In [35]:
from tensorflow.python.training.tracking import tracking

In [ ]:
tracking.AutoTrackable()

In [32]:
trackable.TrackableResource()

AttributeError: module 'tensorflow.python.training.tracking.util' has no attribute 'TrackableResource'

In [226]:
tf.keras.Model.mro()

[tensorflow.python.keras.engine.training.Model,
 tensorflow.python.keras.engine.network.Network,
 tensorflow.python.keras.engine.base_layer.Layer,
 tensorflow.python.module.module.Module,
 tensorflow.python.training.tracking.tracking.AutoTrackable,
 tensorflow.python.training.tracking.base.Trackable,
 tensorflow.python.keras.utils.version_utils.LayerVersionSelector,
 tensorflow.python.keras.utils.version_utils.ModelVersionSelector,
 object]

In [224]:
multilingual_vocab_path = "/space/engineering/pretrained_models/bert/multi_cased_L-12_H-768_A-12/vocab.txt"

In [ ]:
spm.SentencePieceProcessor()

In [ ]:
hub.load()

In [33]:
tf.python.trackable.Trackable()

tensorflow.python.training.tracking.base.Trackable

In [34]:
tf.python.trackable.TrackableReference()

tensorflow.python.training.tracking.base.TrackableReference

In [ ]:
tf.saved_model.Asset()

In [2]:
tf.__version__

'2.2.0'

In [2]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [3]:
tf.config.experimental.list_physical_devices('CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
##restricting no of gpus
gpus = tf.config.experimental.list_physical_devices('GPU')
cpus = tf.config.experimental.list_physical_devices('CPU')
device_to_use = gpus[2]
# device_to_use = []

tf.config.experimental.set_memory_growth(device_to_use,True)
tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
print(tf.config.experimental.get_visible_devices('GPU'))
print(tf.config.experimental.get_visible_devices('CPU'))

[PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [22]:
os.environ["TFHUB_CACHE_DIR"] = '/space/engineering/tfhub_modules'

In [82]:
final_model_path='/space/engineering/tf_serve/models/tryout_jithin'

In [5]:
model_dir='/var/extra/users/jgeorge/tf2.0/input/albert_base'
sp_model_path = os.path.join(model_dir, "assets", "30k-clean.model")
sp = spm.SentencePieceProcessor()
sp.load(sp_model_path)

True

In [51]:
bert_hub_url='https://tfhub.dev/tensorflow/albert_en_large/1'
bert_hub_loaded = hub.load(bert_hub_url)
sentencepiece_path = bert_hub_loaded.sp_model_file.asset_path.numpy()

In [26]:
bert_hub_loaded.sp_model_file.asset_path

<tf.Tensor: shape=(), dtype=string, numpy=b'/space/engineering/tfhub_modules/d0ceaf43f67b8744561ebeeaea4c7c188a6e6f78/assets/30k-clean.model'>

In [30]:
bert_model = hub.KerasLayer(bert_hub_url,trainable=True)
sentencepiece_path = bert_model.resolved_object.sp_model_file.asset_path.numpy()

In [6]:
model_proto = tf.io.gfile.GFile(sp_model_path, 'rb').read()

In [5]:
sp.encode_as_ids("this is it")

[48, 25, 32]

In [6]:
encoded_text1 = tfs.encode(['this is it'],model_file=sp_model_path)
encoded_text1

SentencepieceEncodeDense(values=<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[48, 25, 32]], dtype=int32)>, sequence_length=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>)

In [8]:
tfs.encode(['this is it'],model_proto=model_proto)

SentencepieceEncodeDense(values=<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[48, 25, 32]], dtype=int32)>, sequence_length=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>)

In [9]:
#clear the existing tensorflow graph
tf.keras.backend.clear_session()

In [14]:
[i+1 for i in (range(5))]

[1, 2, 3, 4, 5]

In [1]:
import tensorflow as tf

In [2]:
print('tensorflow version ',tf.__version__)

tensorflow version  2.2.0


In [3]:
##restricting no of gpus
gpus = tf.config.experimental.list_physical_devices('GPU')
device_to_use = gpus[3]
tf.config.experimental.set_memory_growth(device_to_use,True)
tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
print(tf.config.experimental.get_visible_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [208]:
vocab_path='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/vocab_test.txt'

model_dir_lookup='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model/lookup'
model_dir_lookup2 = model_dir_lookup+'_2'

In [20]:
# https://github.com/tensorflow/tensorflow/issues/38305
class VocabLookup(tf.keras.layers.Layer):
    def __init__(self,vocab_path):
        super(VocabLookup, self).__init__(trainable=False,dtype=tf.int32)
        self.vocab_path = vocab_path
    def build(self,input_shape):
        table_init = tf.lookup.TextFileInitializer(self.vocab_path,tf.string,tf.lookup.TextFileIndex.WHOLE_LINE,
                              tf.int64,tf.lookup.TextFileIndex.LINE_NUMBER)
        self.table = tf.lookup.StaticHashTable(table_init,-1)
        self.built=True
        
    def call(self, input_text):
        splitted_text = tf.strings.split(input_text).to_tensor()
        word_ids = self.table.lookup(splitted_text)
        return word_ids
    
    def get_config(self):
        config = super(VocabLookup, self).get_config()
        config.update({'vocab_path': self.vocab_path})
        return config


In [21]:
class VocabLookup2(tf.keras.layers.Layer):
    def __init__(self):
        super(VocabLookup2, self).__init__(trainable=False,dtype=tf.int32)
    def build(self,input_shape):
        self.keys=['hi','testing','lookup','in','tf']
        ##keeping values to start from 1 instead of zero to be consistent with the file based approach
        values=range(1,len(self.keys)+1)
        table_init = tf.lookup.KeyValueTensorInitializer(keys=self.keys,values=values)
        self.table = tf.lookup.StaticHashTable(table_init,-1)
        self.built=True
        
    def call(self, input_text):
        splitted_text = tf.strings.split(input_text).to_tensor()
        word_ids = self.table.lookup(splitted_text)
        return word_ids
    
    def get_config(self):
        config = super(VocabLookup2, self).get_config()
        config.update({'keys': self.keys})
        return config


In [ ]:
VocabLookup()

In [ ]:
tf.saved_model.SaveOptions()

In [17]:
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')

In [22]:
lookup_out = VocabLookup(vocab_path=vocab_path)(input_text)
lookup_out2 = VocabLookup2()(input_text)

In [23]:
model_lookup = tf.keras.Model(inputs={'input_text':input_text},outputs=lookup_out)
model_lookup2 = tf.keras.Model(inputs={'input_text':input_text},outputs=lookup_out2)

In [40]:
process_sentence('talk to an aagent')

words in process sentence [b'talk' b'to' b'an' b'aagent']
per_word_tokenize word  b'talk' shape [] chars [b't' b'a' b'l' b'k']
outer_cond start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)> end <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([4], dtype=int32)>  chars [b't' b'a' b'l' b'k'] is_bad <tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False> sub_tokens <tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([], dtype=object)>
outer_while_loop start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)> end <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([4], dtype=int32)>  chars [b't' b'a' b'l' b'k'] is_bad <tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False> sub_tokens <tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([], dtype=object)>
vocab lookup inside inner_cond  start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([31311, 10114, 10151,    -1])>

In [24]:
print('predict from model1 ', model_lookup.predict(['hi testing lookup in tf randomtext','testing lookup']))
print('predict from model2 ',model_lookup2.predict(['hi testing lookup in tf randomtext']))

predict from model1  [[ 0  1  2  3  4 -1]
 [ 1  2 -1 -1 -1 -1]]
predict from model2  [[ 1  2  3  4  5 -1]]


In [13]:
model_lookup.save(model_dir_lookup)
model_lookup_loaded = tf.keras.models.load_model(model_dir_lookup)
print('loaded model config 1 - \n',model_lookup_loaded.get_config(),'\n')
print('predict from loaded model1 ',model_lookup_loaded.predict(['hi testing lookup in tf randomtext']))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model/lookup/assets
loaded model config 1 - 
 {'name': 'model', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None,), 'dtype': 'string', 'sparse': False, 'ragged': False, 'name': 'input_text'}, 'name': 'input_text', 'inbound_nodes': []}, {'class_name': 'VocabLookup', 'config': {'name': 'vocab_lookup', 'trainable': False, 'dtype': 'int32', 'vocab_path': '/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/vocab_test.txt'}, 'name': 'vocab_lookup', 'inbound_nodes': [[['input_text', 0, 0, {}]]]}], 'input_layers': {'input_text': ['input_text', 0, 0]}, 'output_layers': [['vocab_lookup', 0, 0]]} 

predict from loaded model1  [[ 0  1  2  3  4 -1]]


In [14]:
model_lookup2.save(model_dir_lookup2)
model_lookup_loaded2 = tf.keras.models.load_model(model_dir_lookup2)
print('loaded model config 2 - \n',model_lookup_loaded2.get_config(),'\n')
print('predict from loaded model2 ', model_lookup_loaded2.predict(['hi testing lookup in tf randomtext']))

INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model/lookup_2/assets
loaded model config 2 - 
 {'name': 'model_1', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None,), 'dtype': 'string', 'sparse': False, 'ragged': False, 'name': 'input_text'}, 'name': 'input_text', 'inbound_nodes': []}, {'class_name': 'VocabLookup2', 'config': {'name': 'vocab_lookup2', 'trainable': False, 'dtype': 'int32', 'keys': ['hi', 'testing', 'lookup', 'in', 'tf']}, 'name': 'vocab_lookup2', 'inbound_nodes': [[['input_text', 0, 0, {}]]]}], 'input_layers': {'input_text': ['input_text', 0, 0]}, 'output_layers': [['vocab_lookup2', 0, 0]]} 

predict from loaded model2  [[ 1  2  3  4  5 -1]]


In [ ]:
model_lookup = tf.keras.Model(inputs={'input_text':input_text},outputs=lookup_out2)

In [7]:
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')

In [8]:
input_text2 = tf.keras.Input(shape=(),dtype=tf.string,name='input_text2')

In [9]:
vocab_path='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/vocab.txt'

In [10]:
vocab_model_dir='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model_dir'

In [76]:
class TableLookup(tf.keras.layers.Layer):
    def __init__(self,filepath):
        super(TableLookup,self).__init__(trainable=False,dtype=tf.string)
        self.filepath=filepath
        self.default_label='other-other'
        
    def build(self,input_shape):
        self.built=True
    
    def call(self,label_id):
        table = tf.lookup.TextFileInitializer(self.filepath,
                                      key_dtype=tf.int64,key_index=tf.lookup.TextFileIndex.LINE_NUMBER,
                                     value_dtype=tf.string,value_index=tf.lookup.TextFileIndex.WHOLE_LINE)
# table.initialize(intent_file)
        label_lookup = tf.lookup.StaticHashTable(table
                                        ,default_value=self.default_label)
        return label_lookup.lookup(label_id)

In [83]:
class RegexReplacement(tf.keras.layers.Layer):
    def __init__(self,pattern,replacement):
        super(RegexReplacement,self).__init__(trainable=False,dtype=tf.string)
        self.pattern=pattern
        self.replacement=replacement
        
    def build(self,input_shape):
        self.built=True
    
    def call(self,input_text):
        return tf.strings.regex_replace(input_text,pattern=self.pattern,rewrite=self.replacement)

In [ ]:
# tf.keras.layers.experimental.preprocessing.

In [90]:
tf.keras.layers.Layer.mro()

[tensorflow.python.keras.engine.base_layer.Layer,
 tensorflow.python.module.module.Module,
 tensorflow.python.training.tracking.tracking.AutoTrackable,
 tensorflow.python.training.tracking.base.Trackable,
 tensorflow.python.keras.utils.version_utils.LayerVersionSelector,
 object]

In [ ]:
tf.function()

In [85]:
tf.keras.backend.clear_session()

In [87]:
final_model_path

'/space/engineering/tf_serve/models/tryout_jithin'

In [189]:
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
@tf.function(input_signature=[tf.TensorSpec(shape=None,dtype=tf.string)])
def perform_regex_replace1(input_text):
    replaced_text = RegexReplacement("abc","xyz")(input_text)
    return replaced_text

@tf.function(input_signature=[tf.TensorSpec(shape=None,dtype=tf.string)])
def perform_regex_replace(input_text):
    replaced_text = tf.strings.regex_replace(input_text,pattern="abc",rewrite="xyz")
    return replaced_text
# model_replace = tf.keras.Model(inputs={'input_text':input_text},outputs=replaced_text)

In [221]:
tf.keras.backend.clear_session()

In [222]:
@tf.function(input_signature=[tf.TensorSpec(shape=None,dtype=tf.float32)])
def multiply_x(x):
#     return {'intents':[{'className':tf.constant('dummy'),'score': x*2}]}
    return {'intents': x*2,'intents3':x*3}


In [210]:
multiply_x(2)

{'intents': [{'className': <tf.Tensor: shape=(), dtype=string, numpy=b'dummy'>,
   'score': <tf.Tensor: shape=(), dtype=float32, numpy=4.0>}]}

In [211]:
final_model_path2

'/space/engineering/tf_serve/models/tryout_jithin_2'

In [223]:
tf.saved_model.save(to_export,os.path.join(final_model_path2,'3'),
                    signatures={'serving_default':multiply_x.get_concrete_function(),
                               'web2nl':multiply_x})

INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin_2/3/assets


INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin_2/3/assets


In [196]:
perform_regex_replace.get_concrete_function()

In [207]:
multiply_x.get_concrete_function(tf.TensorSpec(shape=None,dtype=tf.float32))

In [ ]:
multiply_x.get_concrete_function()

In [200]:
multiply_x.get_concrete_function(10)()

<tf.Tensor: shape=(), dtype=int32, numpy=20>

In [ ]:
tf.saved_model.save(to_export,os.path.join(final_model_path2,'2'),
                    signatures={'serving_default':perform_regex_replace.get_concrete_function(),
                               'regex_replace':perform_regex_replace})

In [193]:
tf.keras.Model.mro()

[tensorflow.python.keras.engine.training.Model,
 tensorflow.python.keras.engine.network.Network,
 tensorflow.python.keras.engine.base_layer.Layer,
 tensorflow.python.module.module.Module,
 tensorflow.python.training.tracking.tracking.AutoTrackable,
 tensorflow.python.training.tracking.base.Trackable,
 tensorflow.python.keras.utils.version_utils.LayerVersionSelector,
 tensorflow.python.keras.utils.version_utils.ModelVersionSelector,
 object]

In [192]:
tf.keras.layers.Layer.mro()

[tensorflow.python.keras.engine.base_layer.Layer,
 tensorflow.python.module.module.Module,
 tensorflow.python.training.tracking.tracking.AutoTrackable,
 tensorflow.python.training.tracking.base.Trackable,
 tensorflow.python.keras.utils.version_utils.LayerVersionSelector,
 object]

In [190]:
perform_regex_replace.get_concrete_function()

In [ ]:
perform_regex_replace.get_concrete_function()

In [186]:
final_model_path2='/space/engineering/tf_serve/models/tryout_jithin_2'

In [191]:
tf.saved_model.save(to_export,os.path.join(final_model_path2,'2'),
                    signatures={'serving_default':perform_regex_replace.get_concrete_function(),
                               'regex_replace':perform_regex_replace})

INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin_2/2/assets


INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin_2/2/assets


In [124]:
tf.keras.backend.clear_session()

In [177]:
class Adder(tf.Module):

    @tf.function(input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])
#     @tf.function
    def add(self,x):
        out = x + x + 1
#         out = {'value1': x + x + 1, 'value2': x*2 }
        return out
    
    @tf.function(input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])
    def add_multiply(self,x_mul):
        added = self.add(x_mul)
        return added*5

to_export = Adder()


In [178]:
to_export.add(tf.constant([4.0,5.0]))

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 9., 11.], dtype=float32)>

In [179]:
to_export.add_multiply(tf.constant([4.0,5.0]))

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([45., 55.], dtype=float32)>

In [155]:
to_export.add.get_concrete_function()

In [180]:
to_export.add_multiply.get_concrete_function()

In [166]:
temp = to_export.add.get_concrete_function()
temp

In [181]:
tf.saved_model.save(to_export,os.path.join(final_model_path,'5'),
                    signatures={'serving_default':to_export.add.get_concrete_function(),
                               'add_multiply':to_export.add_multiply.get_concrete_function()})

INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin/5/assets


INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin/5/assets


In [170]:
tf.saved_model.save(to_export,os.path.join(final_model_path,'4'),
                    signatures={'serving_default':to_export.add.get_concrete_function()})

INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin/4/assets


INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin/4/assets


In [134]:
[to_export.add.get_concrete_function(i) for i in range(10)]

In [148]:
final_model_path

'/space/engineering/tf_serve/models/tryout_jithin'

In [ ]:
tf.saved_model.save(to_export,os.path.join(final_model_path,'1'),
                    signatures={'serving_default':to_export.add.get_concrete_function()},
                    options=tf.saved_model.SaveOptions(function_aliases={'add_fn':to_export.add}))

In [158]:
tf.saved_model.save(to_export,os.path.join(final_model_path,'2'),
                    signatures={'serving_default':to_export.add.get_concrete_function()})

INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin/2/assets


INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin/2/assets


In [ ]:
model.save(final_model_path)

### Api call from model

In [81]:
import requests
##python = 3.6.8 
##requests.__version__ = '2.21.0'
print('requests version ',requests.__version__)
requestUrl='http://stable.api.sv2.247-inc.net/v1/machinelearning/predictions/clients/standard/applications/use/models/model'
data_dict = { "signature_name": "serving_default","inputs": {"input_text":["talk to an agent","talk to representative"]}}

response = requests.post(requestUrl,json=data_dict)
output_dict = response.json()
embedding1 = output_dict['outputs'][0] # embedding for 1st sentence
embedding2 = output_dict['outputs'][1] # embedding for 2nd sentence
print('length of embedding1',len(embedding1))

requests version  2.23.0


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
class ApiCall(tf.keras.layers.Layer):
    def __init__(self):
        super(ApiCall,self).__init__(trainable=False,dtype=tf.string)

In [80]:
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')

In [13]:
tfs_encoded = tfs.encode(['this is it','next sentence taks is this random text'],model_proto=model_proto)
tfs_encoded

SentencepieceEncodeDense(values=<tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[   48,    25,    32,     0,     0,     0,     0,     0],
       [  328,  5123, 13302,    18,    25,    48,  5477,  1854]],
      dtype=int32)>, sequence_length=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 8], dtype=int32)>)

In [61]:
tfs_encoded.values.get_shape()

TensorShape([2, 8])

In [51]:
tfs_encoded.values.shape[0]

2

In [69]:
tf.split(tfs_encoded.values,tfs_encoded.values.shape[0],axis=0)

[<tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[48, 25, 32,  0,  0,  0,  0,  0]], dtype=int32)>,
 <tf.Tensor: shape=(1, 8), dtype=int32, numpy=
 array([[  328,  5123, 13302,    18,    25,    48,  5477,  1854]],
       dtype=int32)>]

In [70]:
tf.unstack(tfs_encoded.values,axis=0)

[<tf.Tensor: shape=(8,), dtype=int32, numpy=array([48, 25, 32,  0,  0,  0,  0,  0], dtype=int32)>,
 <tf.Tensor: shape=(8,), dtype=int32, numpy=
 array([  328,  5123, 13302,    18,    25,    48,  5477,  1854],
       dtype=int32)>]

In [53]:
tf.map_fn(lambda x : x, tf.split(tfs_encoded.values,tfs_encoded.values.shape[0],axis=0))

list

In [17]:
tfs_encoded.values

<tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[   48,    25,    32,     0,     0,     0,     0,     0],
       [  328,  5123, 13302,    18,    25,    48,  5477,  1854]],
      dtype=int32)>

In [179]:
tfs_encoded = tfs.encode(['this is it','another sentence'],model_proto=model_proto)
print(tfs_encoded)
# seq_len = 
t = tf.squeeze(tfs_encoded.values)
t

SentencepieceEncodeDense(values=<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[  48,   25,   32],
       [ 226, 5123,    0]], dtype=int32)>, sequence_length=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 2], dtype=int32)>)


<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[  48,   25,   32],
       [ 226, 5123,    0]], dtype=int32)>

In [30]:
[CLS_ID,SEP_ID,PAD_ID]  = tfs.piece_to_id(['[CLS]','[SEP]','<pad>'],model_proto=model_proto)

In [35]:
tf.concat([[CLS_ID],t,[SEP_ID]],axis=0)

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 2, 48, 25, 32,  3], dtype=int32)>

In [184]:
tf.pad(t,[[0,0],[0,2]])

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[  48,   25,   32,    0,    0],
       [ 226, 5123,    0,    0,    0]], dtype=int32)>

In [125]:
tf.expand_dims(t,axis=0)

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[48, 25, 32]], dtype=int32)>

In [176]:
t.shape

TensorShape([3])

In [175]:
t.get_shape()

TensorShape([3])

In [43]:
tf.slice(t,[0],[2])

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([48, 25], dtype=int32)>

In [93]:
[CLS_ID,SEP_ID,PAD_ID]  = tfs.piece_to_id(['[CLS]','[SEP]','<pad>'],model_proto=model_proto)
@tf.function
def get_encoded_text1(input_text_batch,model_proto,max_sequence_length):
#     https://stackoverflow.com/questions/45404056/tf-unstack-with-dynamic-shape
#     batch_size = input_text_batch.shape[0]
#     batch_size = input_text_batch.get_shape()[0]
#     l = tf.split(input_text_batch,batch_size,axis=0)
    
#     l = tf.unstack(input_text_batch,axis=0)
    l = input_text_batch
    tf.print("list l is ",l)
    tf.print("type of list l ",type(l))
    
    
    def process_invidual_line_encoding(x):
        list_x = tf.expand_dims(x,axis=0)
        sp_encoded = tfs.encode(list_x,model_proto=model_proto)
#         sp_encoded = tfs.encode(x)
#         removing the batch dim with size=1
        values = tf.squeeze(sp_encoded.values,name='squeezed_values')
        sequence_length = tf.squeeze(sp_encoded.sequence_length)
        tf.print('squeezed_values ' ,values)
#         trimming to max_length-2 (-2 to incorporate [CLS], [SEP])
        trimmed_max_length = max_sequence_length-2
#         values_trimmed = tf.slice(values,begin=[0],size=[trimmed_max_length],name='trimmed_out')
        values_trimmed = tf.cond(tf.greater(sequence_length,trimmed_max_length), 
                                lambda : tf.slice(values,begin=[0],size=[trimmed_max_length],name='trimmed_out'),lambda : values)
        tf.print('values_trimmed ',values_trimmed)
        concat = tf.concat([[CLS_ID],values_trimmed,[SEP_ID]],axis=0,name='concat_out')
        tf.print('concat_out ',concat)
        tf.print('size of concat ',tf.size(concat))
#         need not prepend anything so 0 for 1st entry in padding, 
#         & next value for padding is how many dimensions required at the end of tensor
        padded = tf.pad(concat,paddings=[[0,max_sequence_length-tf.size(concat)]],name='padded_out')
        return padded
        
    encoded = tf.map_fn(lambda x: process_invidual_line_encoding(x),l,dtype=tf.int32)
    stacked = tf.stack(encoded)
    tf.print('stacked size ',tf.size(stacked))
    tf.print('stacked shape ',tf.shape(stacked))
    return stacked
    

In [11]:
[CLS_ID,SEP_ID,PAD_ID]  = tfs.piece_to_id(['[CLS]','[SEP]','<pad>'],model_proto=model_proto)
# @tf.function
def get_encoded_text(input_text_batch,model_proto,max_sequence_length):
#     https://stackoverflow.com/questions/45404056/tf-unstack-with-dynamic-shape
#     batch_size = input_text_batch.shape[0]
#     batch_size = input_text_batch.get_shape()[0]
#     l = tf.split(input_text_batch,batch_size,axis=0)
    
#     l = tf.unstack(input_text_batch,axis=0)
    l = input_text_batch
    tf.print("list l is ",l)
    tf.print("type of list l ",type(l))
    
    
    def process_invidual_line_encoding(x):
        list_x = tf.expand_dims(x,axis=0)
        sp_encoded = tfs.encode(list_x,model_proto=model_proto)
#         sp_encoded = tfs.encode(x)
#         removing the batch dim with size=1
        values = tf.squeeze(sp_encoded.values,name='squeezed_values')
        sequence_length = tf.squeeze(sp_encoded.sequence_length)
        tf.print('squeezed_values ' ,values)
#         trimming to max_length-2 (-2 to incorporate [CLS], [SEP])
        trimmed_max_length = max_sequence_length-2
#         values_trimmed = tf.slice(values,begin=[0],size=[trimmed_max_length],name='trimmed_out')
        values_trimmed = tf.cond(tf.greater(sequence_length,trimmed_max_length), 
                                lambda : tf.slice(values,begin=[0],size=[trimmed_max_length],name='trimmed_out'),lambda : values)
        tf.print('values_trimmed ',values_trimmed)
        concat = tf.concat([[CLS_ID],values_trimmed,[SEP_ID]],axis=0,name='concat_out')
        tf.print('concat_out ',concat)
        tf.print('size of concat ',tf.size(concat))
        tf.print('shape of concat ',tf.shape(concat))
#         actual_token_length = tf.size(concat) #this would also work since we are processing line by line in this function
        actual_token_length = tf.shape(concat)[-1]
#         need not prepend anything so 0 for 1st entry in padding, 
#         & next value for padding is how many dimensions required at the end of tensor
        padded = tf.pad(concat,paddings=[[0,max_sequence_length-actual_token_length]],name='input_ids')
#         segment_ids = tf.zeros(shape=tf.shape(padded),dtype=tf.int32)
#         or
        segment_ids = tf.zeros_like(padded,dtype=tf.int32,name='segment_ids')
        
        input_mask = tf.scatter_nd(indices=tf.expand_dims(tf.range(0,actual_token_length),axis=1),
                                   updates=tf.ones(shape=[actual_token_length],dtype=tf.int32),
                                   shape=[max_sequence_length],name='input_mask')
        return (padded,segment_ids,input_mask)
#     Issue running map_fn on gpu https://github.com/tensorflow/tensorflow/issues/28007 
#     https://www.tensorflow.org/api_docs/python/tf/device
    with tf.device('/device:CPU:0'):
        encoded = tf.map_fn(lambda x: process_invidual_line_encoding(x),l,dtype=(tf.int32,tf.int32,tf.int32))
    tf.print('encoded map_fn ',encoded)
    tf.print('encoded map_fn type ',type(encoded))
    tf.print('encoded map_fn shape ',tf.shape(encoded))
#     stacked = tf.stack(encoded)
    stacked = encoded
    tf.print('stacked size ',tf.size(stacked))
    tf.print('stacked shape ',tf.shape(stacked))
    return stacked
    

In [38]:
tf.keras.layers.Layer.mro()

[tensorflow.python.keras.engine.base_layer.Layer,
 tensorflow.python.module.module.Module,
 tensorflow.python.training.tracking.tracking.AutoTrackable,
 tensorflow.python.training.tracking.base.Trackable,
 tensorflow.python.keras.utils.version_utils.LayerVersionSelector,
 object]

In [39]:
tf.keras.Model.mro()

[tensorflow.python.keras.engine.training.Model,
 tensorflow.python.keras.engine.network.Network,
 tensorflow.python.keras.engine.base_layer.Layer,
 tensorflow.python.module.module.Module,
 tensorflow.python.training.tracking.tracking.AutoTrackable,
 tensorflow.python.training.tracking.base.Trackable,
 tensorflow.python.keras.utils.version_utils.LayerVersionSelector,
 tensorflow.python.keras.utils.version_utils.ModelVersionSelector,
 object]

In [53]:
model_proto = tf.io.gfile.GFile(sentencepiece_path, 'rb').read()
tftext_spm = tftext.SentencepieceTokenizer(model=model_proto)

In [54]:
tftext_spm.tokenize_with_offsets(['this is a test sentence','next sentence'])

(<tf.RaggedTensor [[48, 25, 21, 1289, 5123], [328, 5123]]>,
 <tf.RaggedTensor [[0, 4, 7, 9, 14], [0, 4]]>,
 <tf.RaggedTensor [[4, 7, 9, 14, 23], [4, 13]]>)

In [56]:
tftext_spm.tokenize(['this is a test sentence','next sentence']).to_tensor()

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[  48,   25,   21, 1289, 5123],
       [ 328, 5123,    0,    0,    0]], dtype=int32)>

In [75]:
tf.keras.layers.Layer()

[tensorflow.python.keras.engine.base_layer.Layer,
 tensorflow.python.module.module.Module,
 tensorflow.python.training.tracking.tracking.AutoTrackable,
 tensorflow.python.training.tracking.base.Trackable,
 tensorflow.python.keras.utils.version_utils.LayerVersionSelector,
 object]

In [ ]:
tf.python.keras.utils.version_utils.LayerVersionSelector()

In [69]:
class SentencepieceTokenization_text(tf.keras.layers.Layer):
    def __init__(self,model_path):
        super(SentencepieceTokenization_text, self).__init__(trainable=False,dtype=tf.int32)
        self.model_path = model_path
        
    def build(self,input_shape):
        self.model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
        self.sp_model = tftext.SentencepieceTokenizer(model=self.model_proto)
#         [self.CLS_ID,self.SEP_ID,self.PAD_ID]  = tfs.piece_to_id(['[CLS]','[SEP]','<pad>'],model_file=self.model_path)
        self.built=True
        
    @tf.function    
    def call(self, input_text):
        encoded_text = self.sp_model.tokenize(input_text).to_tensor()
        ##tensorflow sentence piece works while exporting to graph while, tf_text sentencepiece doesn't
#         encoded_text = tfs.encode(input_text,model_proto=self.model_proto)
        return encoded_text

In [70]:
tf.keras.backend.clear_session()

In [73]:
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
sp_model_path2 = '/space/engineering/tfhub_modules/db96091489ca3b9a526f252d262b880571e1335e/assets/30k-clean.model'
sentence_piece_layer = SentencepieceTokenization_text(model_path=sp_model_path2)
encoded_text = sentence_piece_layer(input_text)
# input_ids,segment_ids,input_mask = sentence_piece_layer(input_text)
# tf.autograph.trace(encoded_text)
# print('input_ids ',input_ids, 'segment_ids ',segment_ids, 'input_mask ',input_mask )
# print('encoded text ',encoded_text)
# model = tf.keras.Model(inputs={'input_text':input_text},outputs=encoded_text)

# model = tf.keras.Model(inputs={'input_text':input_text},outputs=input_ids)
model = tf.keras.Model(inputs={'input_text':input_text},outputs=encoded_text)
sample_text = 'this is a sample sentence'
sample_text2 = 'this is a random text for trying out encoding'
model.summary()
model.predict([sample_text,sample_text2])
# model.sp_model_file = tf.saved_model.Asset(sentence_piece_layer)
# model.save(vocab_model_dir)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_text (InputLayer)      [(None,)]                 0         
_________________________________________________________________
sentencepiece_tokenization_t (None, None)              0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


array([[   48,    25,    21,  5717,  5123,     0,     0,     0,     0],
       [   48,    25,    21,  5477,  1854,    26,   749,    70, 19608]],
      dtype=int32)

In [72]:
sentence_piece_save_path='/var/extra/users/jgeorge/tf2.0/input/dish/models/sentence_piece_text'
model.save(sentence_piece_save_path)

AssertionError: Tried to export a function which references untracked object Tensor("25457:0", shape=(), dtype=resource).TensorFlow objects (e.g. tf.Variable) captured by functions must be tracked by assigning them to an attribute of a tracked object or assigned to an attribute of the main object directly.

In [46]:
# @tf.keras.utils.register_keras_serializable(package="Text")
# checkout tf.module as well (module seems to be super class of Layer)
class SentencepieceTokenization(tf.keras.layers.Layer):
#     def __init__(self,model_proto):
    def __init__(self,model_path):
#         super(SentencepieceTokenization, self).__init__(trainable=False,dynamic=True,dtype=tf.int32)
        super(SentencepieceTokenization, self).__init__(trainable=False,dtype=tf.int32)
        #     self.sp_model_proto = model_proto
#         self.model_path = tf.saved_model.Asset(model_path)
        self.model_path = model_path
        
#         model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
#         self.sp_model = tftext.SentencepieceTokenizer(model=model_proto)
    def build(self,input_shape):
#         self.model_proto = tf.io.gfile.GFile(self.model_path.asset_path, 'rb').read()
        self.model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
#         self.sp_model = tftext.SentencepieceTokenizer(model=self.model_proto)
        [self.CLS_ID,self.SEP_ID,self.PAD_ID]  = tfs.piece_to_id(['[CLS]','[SEP]','<pad>'],model_file=self.model_path)
        self.built=True
        
#     @tf.function    
    def call(self, input_text):
#         encoded_text = self.sp_model.tokenize(input_text).to_tensor()
        ##tensorflow sentence piece works while exporting to graph while, tf_text sentencepiece doesn't
#         encoded_text = tfs.encode(input_text,model_proto=self.model_proto)

#         model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
        encoded_text = self.get_encoded_text(input_text,model_proto=self.model_proto,max_sequence_length=20)
        
#         encoded_text = tf.RaggedTensor.from_tensor(encoded_text.values)
#         return encoded_text.to_tensor()        
        return encoded_text

    @tf.function
    def get_encoded_text(self,input_text_batch,model_proto,max_sequence_length):
        tf.print("input_text_batch is ",input_text_batch)
        tf.print("type of input_text_batch ",type(input_text_batch))

        def process_invidual_line_encoding(x):
#             tf sentencepiece requires a list as input, while the individual value that we get here
#             is a single sentence, so adding one more dimension (i.e adding batch dimension = 1)
            list_x = tf.expand_dims(x,axis=0)
            sp_encoded = tfs.encode(list_x,model_proto=model_proto)
    #         removing the batch dim with size=1 (which we added in the previous step)
            values = tf.squeeze(sp_encoded.values,name='squeezed_values')
            sequence_length = tf.squeeze(sp_encoded.sequence_length)
            tf.print('squeezed_values ' ,values)
    #         trimming to max_length-2 (-2 to incorporate [CLS], [SEP])
            trimmed_max_length = max_sequence_length-2
    #         values_trimmed = tf.slice(values,begin=[0],size=[trimmed_max_length],name='trimmed_out')
            values_trimmed = tf.cond(tf.greater(sequence_length,trimmed_max_length), 
                                    lambda : tf.slice(values,begin=[0],size=[trimmed_max_length],name='trimmed_out'),lambda : values)
            tf.print('values_trimmed ',values_trimmed)
            concat = tf.concat([[self.CLS_ID],values_trimmed,[self.SEP_ID]],axis=0,name='concat_out')
            tf.print('concat_out ',concat)
            tf.print('size of concat ',tf.size(concat))
            tf.print('shape of concat ',tf.shape(concat))
    #         actual_token_length = tf.size(concat) #this would also work since we are processing line by line in this function
            actual_token_length = tf.shape(concat)[-1]
    #         need not prepend anything so 0 for 1st entry in padding, 
    #         & next value for padding is how many dimensions required at the end of tensor
            padded = tf.pad(concat,paddings=[[0,max_sequence_length-actual_token_length]],name='input_ids')
    #         segment_ids = tf.zeros(shape=tf.shape(padded),dtype=tf.int32)
    #         or
            segment_ids = tf.zeros_like(padded,dtype=tf.int32,name='segment_ids')

            input_mask = tf.scatter_nd(indices=tf.expand_dims(tf.range(0,actual_token_length),axis=1),
                                       updates=tf.ones(shape=[actual_token_length],dtype=tf.int32),
                                       shape=[max_sequence_length],name='input_mask')
            return (padded,segment_ids,input_mask)
    #     Issue running map_fn on gpu https://github.com/tensorflow/tensorflow/issues/28007 
    #     https://www.tensorflow.org/api_docs/python/tf/device
        with tf.device('/device:CPU:0'):
            encoded = tf.map_fn(lambda x: process_invidual_line_encoding(x),input_text_batch,dtype=(tf.int32,tf.int32,tf.int32))
        tf.print('encoded map_fn ',encoded)
        tf.print('encoded map_fn type ',type(encoded))
        tf.print('encoded map_fn shape ',tf.shape(encoded))
    #     stacked = tf.stack(encoded)
        stacked = encoded
        tf.print('stacked size ',tf.size(stacked))
        tf.print('stacked shape ',tf.shape(stacked))
        return stacked
    
    
    def get_config(self):
        config = super(SentencepieceTokenization, self).get_config()
        config.update({'model_path': self.model_path})
        return config


In [43]:
tf.keras.backend.clear_session()

In [44]:
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
sp_model_path2 = '/space/engineering/tfhub_modules/db96091489ca3b9a526f252d262b880571e1335e/assets/30k-clean.model'
sentence_piece_layer = SentencepieceTokenization(model_path=sp_model_path2)
# encoded_text = sentence_piece_layer(input_text)
input_ids,segment_ids,input_mask = sentence_piece_layer(input_text)
# tf.autograph.trace(encoded_text)
print('input_ids ',input_ids, 'segment_ids ',segment_ids, 'input_mask ',input_mask )
# print('encoded text ',encoded_text)
# model = tf.keras.Model(inputs={'input_text':input_text},outputs=encoded_text)

model = tf.keras.Model(inputs={'input_text':input_text},outputs=input_ids)
# model = tf.keras.Model(inputs={'input_text':input_text},outputs=encoded_text)
sample_text = 'this is a sample sentence'
sample_text2 = 'this is a random text for trying out encoding'
model.summary()
# model.predict([sample_text,sample_text2])
# model.sp_model_file = tf.saved_model.Asset(sentence_piece_layer)
# model.save(vocab_model_dir)

input_ids  Tensor("sentencepiece_tokenization/Identity:0", shape=(None, None), dtype=int32) segment_ids  Tensor("sentencepiece_tokenization/Identity_1:0", shape=(None, None), dtype=int32) input_mask  Tensor("sentencepiece_tokenization/Identity_2:0", shape=(None, 20), dtype=int32)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_text (InputLayer)      [(None,)]                 0         
_________________________________________________________________
sentencepiece_tokenization ( ((None, None), (None, Non 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.submodules

(<tensorflow.python.keras.engine.input_layer.InputLayer at 0x7fdea8503c50>,
 <__main__.SentencepieceTokenization at 0x7fdea85038d0>)

In [37]:
model.get_layer(index=1)

In [21]:
sp_model_path

'/var/extra/users/jgeorge/tf2.0/input/albert_base/assets/30k-clean.model'

In [29]:
tf.saved_model.

'assets'

In [ ]:
tf.lookup.StaticHashTable()

In [70]:
model_training_output = '/var/extra/users/jgeorge/tf2.0/output/training_out'

In [71]:
model.compile()

In [72]:
summary_dir = os.path.join(model_training_output, 'summaries')
summary_callback = tf.keras.callbacks.TensorBoard(summary_dir)
checkpoint_path = os.path.join(model_training_output, 'checkpoint')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, save_weights_only=True)

In [73]:
custom_callbacks = [summary_callback, checkpoint_callback]

In [44]:
sp_model_filepath = tf.saved_model.Asset(sp_model_path)

In [50]:
sp_model_filepath.asset_path

<tf.Tensor: shape=(), dtype=string, numpy=b'/var/extra/users/jgeorge/tf2.0/input/albert_base/assets/30k-clean.model'>

In [19]:
model.sp_model_filepath = tf.saved_model.Asset(sp_model_path)
model.do_lower_case = tf.saved_model
# model.sp_model_proto = tf.saved_model.Asset()

In [ ]:
tf.saved_model.SaveOptions()

In [68]:
model.save(vocab_model_dir)

INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model_dir/assets


INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model_dir/assets


In [62]:
loaded_model = tf.keras.models.load_model(vocab_model_dir)

In [69]:
loaded_model.predict([sample_text,sample_text2,'another jithin random text to try out'])

input_text_batch is  ["this is a sample sentence" "this is a random text for trying out encoding" "another jithin random text to try out"]
type of input_text_batch  <class 'tensorflow.python.framework.ops.Tensor'>
squeezed_values  [48 25 21 5717 5123]
values_trimmed  [48 25 21 5717 5123]
concat_out  [2 48 25 ... 5717 5123 3]
size of concat  7
shape of concat  [7]
squeezed_values  [48 25 21 ... 749 70 19608]
values_trimmed  [48 25 21 ... 749 70 19608]
concat_out  [2 48 25 ... 70 19608 3]
size of concat  11
shape of concat  [11]
squeezed_values  [226 7022 96 ... 20 1131 70]
values_trimmed  [226 7022 96 ... 20 1131 70]
concat_out  [2 226 7022 ... 1131 70 3]
size of concat  11
shape of concat  [11]
encoded map_fn  ([[2 48 25 ... 0 0 0]
 [2 48 25 ... 0 0 0]
 [2 226 7022 ... 0 0 0]], [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]])
encoded map_fn type  <class 'tuple'>
encoded map_fn shape  [3 3 20]
stacked size  180
stacked

array([[    2,    48,    25,    21,  5717,  5123,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    2,    48,    25,    21,  5477,  1854,    26,   749,    70,
        19608,     3,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    2,   226,  7022,    96,   108,  5477,  1854,    20,  1131,
           70,     3,     0,     0,     0,     0,     0,     0,     0,
            0,     0]], dtype=int32)

In [45]:
model.predict([sample_text,sample_text2])

input_text_batch is  ["this is a sample sentence" "this is a random text for trying out encoding"]
type of input_text_batch  <class 'tensorflow.python.framework.ops.Tensor'>
squeezed_values  [48 25 21 5717 5123]
values_trimmed  [48 25 21 5717 5123]
concat_out  [2 48 25 ... 5717 5123 3]
size of concat  7
shape of concat  [7]
squeezed_values  [48 25 21 ... 749 70 19608]
values_trimmed  [48 25 21 ... 749 70 19608]
concat_out  [2 48 25 ... 70 19608 3]
size of concat  11
shape of concat  [11]
encoded map_fn  ([[2 48 25 ... 0 0 0]
 [2 48 25 ... 0 0 0]], [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]])
encoded map_fn type  <class 'tuple'>
encoded map_fn shape  [3 2 20]
stacked size  120
stacked shape  [3 2 20]


array([[    2,    48,    25,    21,  5717,  5123,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    2,    48,    25,    21,  5477,  1854,    26,   749,    70,
        19608,     3,     0,     0,     0,     0,     0,     0,     0,
            0,     0]], dtype=int32)

In [ ]:
tf.keras.layers.Embedding()

In [128]:
t = tf.zeros([10])
t

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [126]:
tf.range(0,limit=5)

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4], dtype=int32)>

In [142]:
t1 = tf.reshape(tf.range(0,5),shape=(5,1))
t1

<tf.Tensor: shape=(5, 1), dtype=int32, numpy=
array([[0],
       [1],
       [2],
       [3],
       [4]], dtype=int32)>

In [147]:
tf.shape(t1)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([5, 1], dtype=int32)>

In [150]:
tf.reverse(tf.shape(t1),axis=[0])

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 5], dtype=int32)>

In [141]:
tf.scatter_nd(indices=tf.reshape(tf.range(0,5),shape=(5,1)),updates=tf.ones(shape=[5]),shape=[10])

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.], dtype=float32)>

In [139]:
tf.range(0,5)

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4], dtype=int32)>

In [140]:
tf.reshape(tf.range(0,5),shape=(5,1))

<tf.Tensor: shape=(5, 1), dtype=int32, numpy=
array([[0],
       [1],
       [2],
       [3],
       [4]], dtype=int32)>

In [152]:
tf.expand_dims(tf.range(0,5),axis=1)

<tf.Tensor: shape=(5, 1), dtype=int32, numpy=
array([[0],
       [1],
       [2],
       [3],
       [4]], dtype=int32)>

In [165]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_text (InputLayer)      [(None,)]                 0         
_________________________________________________________________
sentencepiece_tokenization_3 SentencepieceEncodeDense( 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [24]:
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
input_text

<tf.Tensor 'input_text_1:0' shape=(None,) dtype=string>

In [157]:
input_text

<tf.Tensor 'input_text_1:0' shape=(None, None) dtype=string>

In [139]:
sample_text = 'this is a sample sentence'
sample_text2 = 'this is a random text for trying out encoding'
model.predict([sample_text,sample_text2])

SentencepieceEncodeDense(values=array([[   48,    25,    21,  5717,  5123,     0,     0,     0,     0],
       [   48,    25,    21,  5477,  1854,    26,   749,    70, 19608]],
      dtype=int32), sequence_length=array([5, 9], dtype=int32))

In [143]:
sample_text = 'this is a sample sentence'
model.predict([[sample_text]])

SentencepieceEncodeDense(values=array([[  48,   25,   21, 5717, 5123]], dtype=int32), sequence_length=array([5], dtype=int32))

In [14]:
encoded_text4 = VocabLookup(vocab_path)(input_text)

NameError: name 'VocabLookup' is not defined

In [166]:
model4 = tf.keras.Model(inputs={'input_text':input_text},outputs=encoded_text4)

In [168]:
model4.save(vocab_model_dir,include_optimizer=False,save_format='tf')

INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model_dir/assets


In [ ]:
encoded_text2 = tfs.encode([input_text],model_proto=model_proto)

In [29]:
sentence_piece_tokenizer = tftext.SentencepieceTokenizer(model=model_proto)

In [33]:
tokenized_out_ragged =  sentence_piece_tokenizer.tokenize(['this is it','next sentence taks is this random text'])

In [34]:
tokenized_out_ragged.to_tensor()

<tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[   48,    25,    32,     0,     0,     0,     0,     0],
       [  328,  5123, 13302,    18,    25,    48,  5477,  1854]],
      dtype=int32)>

In [45]:
tfs_encoded = tfs.encode(['this is it','next sentence taks is this random text'],model_proto=model_proto)
tfs_ragged = tf.RaggedTensor.from_tensor(tfs_encoded.values)
tfs_ragged

<tf.RaggedTensor [[48, 25, 32, 0, 0, 0, 0, 0], [328, 5123, 13302, 18, 25, 48, 5477, 1854]]>

In [ ]:
sentence_piece_tokenizer()

In [ ]:
tf.keras.layers.Layer()

In [23]:
replaced_text = RegexReplacement("abc","xyz")(input_text)

In [24]:
model_regex_replace = tf.keras.Model(inputs={"input_text":input_text},outputs=replaced_text)

In [27]:
predicted_replaced_text = model_regex_replace.predict([['this is it abc']])

predicted_replaced_text

array([b'this is it xyz'], dtype=object)

In [9]:
regex_replaced_text2 = RegexReplacement("this","that google")(input_text2) 

In [36]:
# encoded_text = SentencepieceTokenization(model_proto=model_proto)(input_text)
sentence_piece_layer = SentencepieceTokenization(model_path=sp_model_path)
encoded_text = sentence_piece_layer(input_text)
encoded_text2 = sentence_piece_layer(regex_replaced_text2)

In [37]:
model = tf.keras.Model(inputs={'input_text':input_text},outputs=encoded_text)
model2 = tf.keras.Model(inputs={'input_text':input_text2},outputs=encoded_text2)

In [13]:
sample_text = 'this is a sample sentence'
sample_text2 = 'that google is a sample sentence'


In [40]:
model.predict([[sample_text]])

array([[  48,   25,   21, 5717, 5123]], dtype=int32)

In [38]:
model.save(vocab_model_dir)

AssertionError: Tried to export a function which references untracked object Tensor("573:0", shape=(), dtype=resource).TensorFlow objects (e.g. tf.Variable) captured by functions must be tracked by assigning them to an attribute of a tracked object or assigned to an attribute of the main object directly.

In [14]:
model.predict([[sample_text2]])

SentencepieceEncodeDense(values=array([[  30, 8144,   25,   21, 5717, 5123]], dtype=int32), sequence_length=array([6], dtype=int32))

In [16]:
model2.predict([[sample_text]])

SentencepieceEncodeDense(values=array([[  30, 8144,   25,   21, 5717, 5123]], dtype=int32), sequence_length=array([6], dtype=int32))

In [21]:
tfs.encode([sample_text2],model_proto=model_proto)

SentencepieceEncodeDense(values=<tf.Tensor: shape=(1, 6), dtype=int32, numpy=array([[  30, 8144,   25,   21, 5717, 5123]], dtype=int32)>, sequence_length=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([6], dtype=int32)>)

In [38]:
input_number = tf.keras.Input(shape=(),dtype=tf.int32,name='input_number')

In [46]:
class Addition(tf.keras.layers.Layer):
    def __init__(self):
        super(Addition, self).__init__(trainable=False,dtype=tf.int32)
        #     self.sp_model_proto = model_proto
#         self.sp_model = tftext.SentencepieceTokenizer(model=model_proto)
    
    def call(self, input_number):
        
        return input_number + 3

In [47]:
modified_number = Addition()(input_number)

In [48]:
model2 = tf.keras.Model(inputs={'input_number':input_number},outputs=modified_number)

In [50]:
model2.predict([3,5])

array([6, 8], dtype=int32)

In [79]:
predicted_output_ragged = model3.predict([['this is it']])

In [80]:
predicted_output_ragged

<tf.RaggedTensor [[48, 25, 32]]>

In [81]:
predicted_output_ragged.to_tensor()

<tf.Tensor: id=1038, shape=(1, 3), dtype=int32, numpy=array([[48, 25, 32]], dtype=int32)>

In [91]:
model.inputs

[<tf.Tensor 'input_text_2:0' shape=(None,) dtype=string>]

In [16]:
model_export_dir = '/var/extra/users/jgeorge/tf2.0/input/albert_base_sentencepiece'

In [33]:
#https://github.com/tensorflow/tensorflow/issues/31300
model.save(model_export_dir,include_optimizer=False,save_format='tf')

INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/albert_base_sentencepiece/assets


In [12]:
tf.saved_model.save(model,export_dir=model_export_dir)

AssertionError: Tried to export a function which references untracked object Tensor("StatefulPartitionedCall/args_1:0", shape=(), dtype=resource).TensorFlow objects (e.g. tf.Variable) captured by functions must be tracked by assigning them to an attribute of a tracked object or assigned to an attribute of the main object directly.

In [77]:
vocab_path

'/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/vocab.txt'

In [78]:
table = tf.lookup.StaticHashTable(tf.lookup.TextFileInitializer(vocab_path,tf.string,tf.lookup.TextFileIndex.WHOLE_LINE,
                              tf.int64,tf.lookup.TextFileIndex.LINE_NUMBER),0)

In [79]:
table.lookup(['hi','this'])

AttributeError: 'list' object has no attribute 'dtype'

In [ ]:
tf.lookup.TextFileInitializer()

In [142]:
tf.strings.split('hi this is jithin')

<tf.Tensor: id=5356, shape=(4,), dtype=string, numpy=array([b'hi', b'this', b'is', b'jithin'], dtype=object)>

In [135]:
lookup_out1 = VocabLookup(vocab_path=vocab_path)(splitted_text)

In [136]:
model_lookup1 = tf.keras.Model(inputs={'splitted_text':splitted_text},outputs=lookup_out1)

In [85]:
table = tf.lookup.StaticHashTable(tf.lookup.TextFileInitializer(vocab_path,tf.string,tf.lookup.TextFileIndex.WHOLE_LINE,
                              tf.int64,tf.lookup.TextFileIndex.LINE_NUMBER),1)

In [153]:
lookup_out = VocabLookup(vocab_path=vocab_path)(input_text)

In [138]:
model_lookup1.predict([['hi','this','is','random']])

array([[1],
       [2],
       [3],
       [0]])

In [155]:
model_lookup.predict([['hi this is asdfasdf']])

array([[1, 2, 3, 0]])

In [139]:
model_lookup_export_dir = '/var/extra/users/jgeorge/tf2.0/input/albert_base_lookup'

In [141]:
model_lookup.save(model_lookup_export_dir,include_optimizer=False,save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/albert_base_lookup/assets


In [ ]:
model_lookup